# rohe beschreibung was in den codes gemacht wird und wieso

oft wird rag gebaut mit pdfs, und wenn man so ein system aufbaut kann man sicherlich eine grössere user base ansprechen, welche das system dann auch bauen möchten. in diesem projekt möchte ich aber webscraping von yahoo finance machen, weil viele leute auch yahoo finance lesen und auf yahoo finance viel mehr informationen sind. ich kann dort webscraping machen von bestimmten aktien, fonds oder indexes und dann alle informationen scrapen von kursen, artikeln, stock prices,
intraday trading data,
option prices;
company disclosures and information such as :
financial information,
earnings,
analyst ratings and recommendations,
insider trades,
SEC filings,
news,
large shareholders;

 dann kann ich all das in einen ordner speichern und einen rag bauen auf all diese sachen. das spart einerseits zeit weil man nicht manuell nach pdfs online suchen muss zu den gewünschten themen, sondern man kann alles zeitsparend an einem ort scrapen und dann weiterbenutzen als input für das rag.

 denn technisch gesehen macht LangChain keinen Unterschied, ob deine Daten ursprünglich aus PDFs, TXTs, Webseiten oder JSON stammen. Wichtig ist nur, was in den vektorstore eingespeist wird, ist reiner text, nicht das ursprüngliche Dateiformat (wie z.b. pdf).

 Ob du folgenden Text aus einem PDF extrahierst:
 In 2023, Allianz reported a revenue of €152.7 billion and a net income of €14.2 billion.

oder denselben Text aus einem JSON-Feld zusammenstellst wie:
{ "revenue": "€152.7 billion", "net_income": "€14.2 billion" }

ist für den Vektorstore und LangChain völlig egal, solange du am Ende diesen Text als Chunk übergibst.

📘 Warum sieht man online meist PDFs?

PDFs sind realistische Quellen (z. B. Geschäftsberichte, Studien)
Sie demonstrieren komplexere Verarbeitung (PDF → Text → Chunk → Embedding)
Viele Nutzer haben genau solche Dateien → Tutorials wollen für die breite Masse relevant sein
JSON ist oft maschinenlesbar, aber in der Praxis weniger "typisch" für Endnutzer
✅ Wann ist JSON sogar besser?

Wenn du strukturierte, tabellarische Daten hast (Finanzdaten, ESG-KPIs, Zeitreihen)
Wenn du Daten aus APIs sammelst (z. B. von Morningstar, AlphaVantage, Unternehmensdatenbanken)
Wenn du dein System voll automatisiert aufbaust ohne manuelles PDF-Verarbeiten
Wenn du bestimmte Felder gezielt extrahieren und in Textform bringen willst

Ja, du kannst problemlos ein LangChain-basiertes RAG-System mit JSON-Dateien aufbauen.
🧱 Die einzige Voraussetzung: Du musst den Inhalt in Klartext-Chunks bringen, damit das LLM damit arbeiten kann.

für jede firma und für jeden datensatz (aktienkurs, cashflow etc.) wird jeweils ein json datei generiert aus folgendem grund:

1. Modular & skalierbar

Wenn jede Bank ihr eigenes Verzeichnis bzw. JSON-Set hat, kannst du:

gezielt einzelne Firmen updaten (nur JPM aktualisieren statt alles neu scrapen)
schneller auf firmenspezifische Daten zugreifen
einfacher firmenspezifische Claims prüfen (z. B. „Wie war der Cashflow von JPM?“)

2. Sauberer für Chunking & RAG

Ein RAG-System arbeitet oft dokumentenbasiert. Wenn jede Firma ein „Dokument“ bzw. Datensatz ist, kannst du:

firmenspezifische Chunks erzeugen
Kontext klar steuern: "Context: JPMorgan" → keine Gefahr durch „vermischte“ Daten
Retrieval filtern nach company_name, ticker, etc.

3. Einfachere Annotation / Fact-Checking

Für deinen Fragebogen willst du Claims wie:

„JPMorgan hatte 2024 einen positiven Free Cash Flow.“
→ Da ist es superpraktisch, wenn du direkt auf JPM/cashflow.json zugreifen kannst – statt alles aus einem Riesendokument zu parsen.

# source .venv/bin/activate

In [53]:
# imports

import yfinance as yf
import pandas as pd
import matplotlib
from datetime import datetime
from pytz import timezone
import os
import json
import requests


In [58]:
# masterfile, das alles webscraped mit yfinance und in einzelnen json files pro unternehmen abspeichert


tickers = ['JPM', 'BAC', 'C', 'UBSG.SW', 'HSBA.L', 'BNP.PA']

filename_prefixes = {
    'JPM': 'jpm_',
    'BAC': 'bac_',
    'C': 'citi_',
    'UBSG.SW': 'ubsg_',
    'HSBA.L': 'hsbc_',
    'BNP.PA': 'bnp_'
}

folder_names = {
    'JPM': 'jpm',
    'BAC': 'bac',
    'C': 'citi',
    'UBSG.SW': 'ubs',
    'HSBA.L': 'hsbc',
    'BNP.PA': 'bnp'
}

base_path = "data"

# save json files with datetime
def save_json(data, path):
    def json_converter(o):
        if isinstance(o, (datetime.datetime, datetime.date)):
            return o.isoformat()
        return str(o)

    try:
        if isinstance(data, pd.DataFrame):
            if data.empty:
                print(f"Empty DataFrame: {path}")
                return
            data = data.reset_index()
            data.columns = [str(col) for col in data.columns]
            for col in data.columns:
                if pd.api.types.is_datetime64_any_dtype(data[col]):
                    data[col] = pd.to_datetime(data[col]).dt.strftime('%Y-%m-%d')
            data = data.to_dict(orient="records")
        elif isinstance(data, (dict, list)):
            pass
        else:
            print(f"No serialization: {path}")
            return

        with open(path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False, default=json_converter)

        print(f"Saved in: {path}")
    except Exception as e:
        print(f"Error while saving {path}: {e}")

# get and save data for each ticker
for ticker in tickers:
    print(f"\n Worked: {ticker}")
    prefix = filename_prefixes[ticker]
    folder = folder_names[ticker]
    folder_path = os.path.join(base_path, folder)
    os.makedirs(folder_path, exist_ok=True)

    try:
        # stock prices via yf.download
        hist = yf.download(ticker, group_by="ticker", auto_adjust=False)
        hist.reset_index(inplace=True)
        hist['Date'] = pd.to_datetime(hist['Date']).dt.strftime('%Y-%m-%d')
        save_json(hist, os.path.join(folder_path, f"{prefix}shareprices.json"))
    except Exception as e:
        print(f"shareprices error {ticker}: {e}")

    try:
        
        t = yf.Ticker(ticker)

        # Informations
        save_json(t.info, os.path.join(folder_path, f"{prefix}info.json"))

        # News
        save_json(t.news, os.path.join(folder_path, f"{prefix}news.json"))

        # Earnings
        save_json(t.earnings_dates, os.path.join(folder_path, f"{prefix}earning_dates.json"))

        # Insider Transactions
        save_json(t.insider_transactions, os.path.join(folder_path, f"{prefix}insider_transactions.json"))

        # Institutional Holders
        save_json(t.institutional_holders, os.path.join(folder_path, f"{prefix}institutional_holders.json"))

        # Mutual Fund Holders
        save_json(t.mutualfund_holders, os.path.join(folder_path, f"{prefix}mutualfund_holders.json"))

        # Cashflow
        save_json(t.cashflow.transpose(), os.path.join(folder_path, f"{prefix}cashflow.json"))

        # Balance Sheet
        save_json(t.balance_sheet.transpose(), os.path.join(folder_path, f"{prefix}balance_sheet.json"))

        # Financial Data
        save_json(t.financials.transpose(), os.path.join(folder_path, f"{prefix}financials.json"))

    except Exception as e:
        print(f"ticker error {ticker}: {e}")



 Worked: JPM


[*********************100%***********************]  1 of 1 completed


Saved in: data/jpm/jpm_shareprices.json
Saved in: data/jpm/jpm_info.json
Saved in: data/jpm/jpm_news.json
Saved in: data/jpm/jpm_earning_dates.json
Saved in: data/jpm/jpm_insider_transactions.json
Saved in: data/jpm/jpm_institutional_holders.json
Saved in: data/jpm/jpm_mutualfund_holders.json
Saved in: data/jpm/jpm_cashflow.json
Saved in: data/jpm/jpm_balance_sheet.json
Saved in: data/jpm/jpm_financials.json

 Worked: BAC


[*********************100%***********************]  1 of 1 completed


Saved in: data/bac/bac_shareprices.json
Saved in: data/bac/bac_info.json
Saved in: data/bac/bac_news.json
Saved in: data/bac/bac_earning_dates.json
Saved in: data/bac/bac_insider_transactions.json
Saved in: data/bac/bac_institutional_holders.json
Saved in: data/bac/bac_mutualfund_holders.json
Saved in: data/bac/bac_cashflow.json
Saved in: data/bac/bac_balance_sheet.json
Saved in: data/bac/bac_financials.json

 Worked: C


...

Saved in: data/ubs/ubsg_shareprices.json
Saved in: data/ubs/ubsg_info.json
Saved in: data/ubs/ubsg_news.json
Saved in: data/ubs/ubsg_earning_dates.json
Empty DataFrame: data/ubs/ubsg_insider_transactions.json
Saved in: data/ubs/ubsg_institutional_holders.json
Saved in: data/ubs/ubsg_mutualfund_holders.json
Saved in: data/ubs/ubsg_cashflow.json
Saved in: data/ubs/ubsg_balance_sheet.json
Saved in: data/ubs/ubsg_financials.json

 Worked: HSBA.L


[*********************100%***********************]  1 of 1 completed


Saved in: data/hsbc/hsbc_shareprices.json
Saved in: data/hsbc/hsbc_info.json
Saved in: data/hsbc/hsbc_news.json
Saved in: data/hsbc/hsbc_earning_dates.json
Saved in: data/hsbc/hsbc_insider_transactions.json
Saved in: data/hsbc/hsbc_institutional_holders.json
Saved in: data/hsbc/hsbc_mutualfund_holders.json
Saved in: data/hsbc/hsbc_cashflow.json
Saved in: data/hsbc/hsbc_balance_sheet.json
Saved in: data/hsbc/hsbc_financials.json

 Worked: BNP.PA


[*********************100%***********************]  1 of 1 completed


Saved in: data/bnp/bnp_shareprices.json
Saved in: data/bnp/bnp_info.json
Saved in: data/bnp/bnp_news.json
Saved in: data/bnp/bnp_earning_dates.json
Empty DataFrame: data/bnp/bnp_insider_transactions.json
Saved in: data/bnp/bnp_institutional_holders.json
Saved in: data/bnp/bnp_mutualfund_holders.json
Saved in: data/bnp/bnp_cashflow.json
Saved in: data/bnp/bnp_balance_sheet.json
Saved in: data/bnp/bnp_financials.json


In [ ]:
# FAILED TRY OUT: artikel scrapen online und alle titel und url abspeichern in einem einzigen json file pro unternehmen

import requests
import json
import time
import os

def search_yahoo_news_by_keyword(keyword, max_articles=1000, step=20):
    base_url = "https://query1.finance.yahoo.com/v1/finance/search"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    all_articles = []
    start = 0

    while len(all_articles) < max_articles:
        params = {
            "q": keyword,
            "newsCount": step,
            "start": start
        }

        res = requests.get(base_url, headers=headers, params=params)
        if res.status_code != 200:
            print(f"getting data error (Statuscode {res.status_code})")
            break

        data = res.json()
        news_batch = data.get("news", [])
        if not news_batch:
            print("no more articles found.")
            break

        for item in news_batch:
            title = item.get("title", "")
            if keyword.lower() in title.lower():
                all_articles.append({
                    "title": title,
                    "url": item.get("link"),
                    "publisher": item.get("publisher"),
                    "providerPublishTime": item.get("providerPublishTime")
                })

        print(f"🔎 {len(all_articles)} found articles...")
        start += step
        time.sleep(1)

    return all_articles[:max_articles]

def save_news_as_json(news_list, keyword):
    folder = "yahoo_news_filtered"
    os.makedirs(folder, exist_ok=True)
    file_path = os.path.join(folder, f"{keyword}_filtered_news.json")

    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(news_list, f, ensure_ascii=False, indent=2)

    print(f"✅ {len(news_list)} save articles in: {file_path}")

# search for this key word
keyword = input("UBS").strip()

# get and save article
filtered_news = search_yahoo_news_by_keyword(keyword, max_articles=1000)
save_news_as_json(filtered_news, keyword)

# ------------------------------
# url nehmen und dann den text extrahieren und speichern
from newspaper import Article

def extract_article_content(url):
    article = Article(url)
    article.download()
    article.parse()

    return {
        "title": article.title,
        "text": article.text
    }

# Beispiel-URL (ersetzen durch deine eigene)
url = input("enter article url: ").strip()

try:
    result = extract_article_content(url)
    print("\ntitle:")
    print(result["title"])
    print("\narticleText:")
    print(result["text"])
except Exception as e:
    print("error while extracting article:", e)



# diese idee wurde aber verworfen, denn als ich 1000 artikel scrapen wollte kamen nur ca 20 raus von einem einzigen tag und der rest waren einfach kopien.
# scraping recht eingeschränkt. darum mache ich das nun manuell und somit kann ich alles holen was ich effektiv brauche. 
# der fokus der arbeit liegt ja bei fact checking llm darum hole ich lieber manuell gute daten und weiss dass diese stimmen und kann dann einen besseren benchmark haben 
# und bessere fragen stellen für fragebogen mit besseren antwortgenauigkeit das ist das wichtigste um die LLMs dann zu fact checken

In [59]:
# scrape wikipedia sites

import requests
import json
import os
from datetime import datetime

def get_wikipedia_article(title, lang="en"):

    url = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts|info",
        "titles": title,
        "explaintext": True,
        "inprop": "url"
    }

    res = requests.get(url, params=params)
    data = res.json()
    pages = data["query"]["pages"]
    page = next(iter(pages.values()))

    article_info = {
        "title": page.get("title"),
        "url": page.get("fullurl"),
        "extract": page.get("extract"),
        "last_updated": datetime.utcnow().isoformat()
    }

    return article_info

def save_article_to_json(article_info, folder_path, filename):

    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path, filename)

    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(article_info, f, ensure_ascii=False, indent=2)

    print(f"saved: {file_path}")

folder_names = {
    'JPM': 'jpm',
    'BAC': 'bac',
    'C': 'citi',
    'UBSG.SW': 'ubs',
    'HSBA.L': 'hsbc',
    'BNP.PA': 'bnp'
}

file_prefixes = {
    'JPM': 'jpm_',
    'BAC': 'bac_',
    'C': 'citi_',
    'UBSG.SW': 'ubsg_',
    'HSBA.L': 'hsbc_',
    'BNP.PA': 'bnp_'
}

banks = [
    {"wikipedia_title": "Bank of America", "ticker": "BAC"},
    {"wikipedia_title": "BNP Paribas", "ticker": "BNP.PA"},
    {"wikipedia_title": "Citigroup", "ticker": "C"},
    {"wikipedia_title": "HSBC", "ticker": "HSBA.L"},
    {"wikipedia_title": "JPMorgan Chase", "ticker": "JPM"},
    {"wikipedia_title": "UBS", "ticker": "UBSG.SW"}
]

for bank in banks:
    ticker = bank["ticker"]
    print(f"Get articles for: {bank['wikipedia_title']}")

    try:
        article = get_wikipedia_article(bank["wikipedia_title"], lang="en")

        folder_path = os.path.join("data", folder_names[ticker])
        filename = file_prefixes[ticker] + "wikipedia.json"

        save_article_to_json(
            article_info=article,
            folder_path=folder_path,
            filename=filename
        )
    except Exception as e:
        print(f"Error with {bank['wikipedia_title']}: {e}")


Get articles for: Bank of America


/var/folders/jb/lln_4j7d4976cj9dpf8gkcsh0000gn/T/ipykernel_2037/1691552168.py:29: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "last_updated": datetime.utcnow().isoformat()


saved: data/bac/bac_wikipedia.json
Get articles for: BNP Paribas
saved: data/bnp/bnp_wikipedia.json
Get articles for: Citigroup
saved: data/citi/citi_wikipedia.json
Get articles for: HSBC
saved: data/hsbc/hsbc_wikipedia.json
Get articles for: JPMorgan Chase
saved: data/jpm/jpm_wikipedia.json
Get articles for: UBS
saved: data/ubs/ubsg_wikipedia.json
